In [1]:
# run this same experiment again but instead of 2,0,0 -> 0,0,2 have 2,0,0 -> 0,1,1 this would be cool 
# declaring all dependancies    
import qutip as qt
import numpy as np
from quantum_logical.pulsesim import QuantumSystem, Pulse
from quantum_logical.pulsesim.mode import QubitMode, SNAILMode, CavityMode
from quantum_logical.pulsesim.build_hamiltonian import Build_hamiltonian
import matplotlib.pyplot as plt
from quantum_logical.channel import AmplitudeDamping, PhaseDamping
from quantum_logical.trotter import TrotterGroup
from quantum_logical.interaction import ConversionGainInteraction
from qutip import Options
from tqdm import tqdm
from progressbar import progressbar
from threading import Thread
from multiprocessing.pool import Pool, ThreadPool
from numpy.linalg import eig
from quantum_logical.creating_operators import Creating_operators
from quantum_logical.operators import selective_destroy

# opts = Options(nsteps=1e6, atol=1e-8, rtol=1e-6)
# p_bar = qt.ui.TextProgressBar()  # None
opts = {"nsteps": 1e6, "atol": 1e-8, "rtol": 1e-6, "progress_bar": "text"}

In [2]:
fund_ops_list = []
H = Creating_operators(qubit_count=2, qubit_dim=3)
fund_ops_list.append(H.create_ops_unconventional_annhilation(from_level=0, to_level=1))  # this is not necessary with the new addition for the operator 
fund_ops_list.append(H.create_ops_unconventional_annhilation(from_level=1, to_level=2))

# # list of operators based on the fundamental single qubit operators 
# ops_list = []
# ops_list.append(qt.tensor(fund_ops_list[0], fund_ops_list[1].dag()))
# ops_list.append(qt.tensor(fund_ops_list[1], fund_ops_list[0].dag()))
# ops_list.append(qt.tensor(fund_ops_list[1].dag(), fund_ops_list[0]))
# ops_list.append(qt.tensor(fund_ops_list[0].dag(), fund_ops_list[1]))

# list of operators based on the full dimension operators
ops_list = []
ops_list.append(H.creating_ops_full_dim(qubit1_from=1, qubit1_to=0, qubit2_from=1, qubit2_to=2))
ops_list.append(H.creating_ops_full_dim(qubit1_from=2, qubit1_to=1, qubit2_from=0, qubit2_to=1))
ops_list.append(H.creating_ops_full_dim(qubit1_from=1, qubit1_to=2, qubit2_from=1, qubit2_to=0))
ops_list.append(H.creating_ops_full_dim(qubit1_from=0, qubit1_to=1, qubit2_from=2, qubit2_to=1))

ops_list1 = []
ops_list1.append(H.create_ops_unconventional_annhilation(from_level=1, to_level=0))

# this doesnt work yet but it is necessary for this to work
# ops_list2 = []
# ops_list1 = np.zeros((9,9), dtype=complex)
# for i in range(len(ops_list)):
#     for j in range(ops_list[i].shape[0]):
#         for k in range(ops_list[i].shape[0]):
#             ops_list1[j,k] = ops_list[i][j,k]
#     ops_list2.append(ops_list1)

# for i in range(len(ops_list2)):
#     ops_list2[i] = qt.Qobj(ops_list2[i])

# ops_list[0].shape[0]


In [3]:
selective_destroy(levels=3, from_level=1, to_level=0) == ops_list1[0]

True

In [9]:
# put in the target cnot 
# check that these operations give a cnot
H_new = ConversionGainInteraction(gg=np.pi / 4, gc= np.pi / 4, transmon_levels=3)
U_cnot_no_time = H_new.construct_U(1.0)
# H_mine = (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0].dag()) + (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0]) + (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0].dag()).dag() + (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0]).dag()
# U_mine = H.propagator(H=H_mine, T=1)
# U_mine

In [10]:
U_cnot_no_time

Quantum object: dims=[[3, 3], [3, 3]], shape=(9, 9), type='oper', dtype=Dense, isherm=False
Qobj data =
[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.        -0.70710678j 0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678+0.j         0.        +0.j
  0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         1.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.        +0.j
  0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j         0.70710678+0.j  

In [11]:
# make the table of coefficients and then create the pulse object (smooth box is probably the best idea)
gg_coeff = gc_coeff = np.linspace(0, 2 * np.pi, 10)
time_coeff = np.linspace(10,100,10)
coeffs = [(i, j, k) for i in gg_coeff for j in gc_coeff for k in time_coeff]
# create the gain and conversion operators 

# def sim_task_gate(coeffs):
#      gg, gc, time = coeffs

#      tlist = np.linspace(0, time, 20)
#      # build the pulse
#      args = {"shape": Pulse.box, "shape_params": {"t0": time * .05, "width": time * .9}}
#      pulse1 = Pulse(amp = gg)
#      pulse2 = Pulse(amp = gc)

#      H = [[qt.tensor(ops_list1[0], ops_list1[0].dag()), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]), pulse2.drive],
#           [qt.tensor(ops_list1[0], ops_list1[0].dag()).dag(), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]).dag(), pulse2.drive]]

#      U_prop_target = qt.propagator(H=H, t=tlist, args=args)

#      fid = qt.process_fidelity(U_prop_target, U_cnot_no_time)

#      return fid 

# results = qt.parallel.parallel_map(task=sim_task_gate, values=coeffs, progress_bar=True)
fid = []

for i in tqdm(coeffs):
     tlist = np.linspace(0, i[2], 20)
     # build the pulse
     args = {"shape": Pulse.box, "shape_params": {"t0": i[2] * .05, "width": i[2] * .9}}
     pulse1 = Pulse(omega=0, amp = i[1])
     pulse2 = Pulse(omega=0, amp = i[0])

     H = [[qt.tensor(ops_list1[0], ops_list1[0].dag()), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]), pulse2.drive],
          [qt.tensor(ops_list1[0], ops_list1[0].dag()).dag(), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]).dag(), pulse2.drive]]

     U_prop_target = qt.propagator(H=H, t=tlist, args=args)

     fid.append(qt.process_fidelity(U_prop_target, U_cnot_no_time))


# how can fidelity be best measured between the propagators and make sure that you are not getting inappropriate values. 
# what is the frequency of the pulse?


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:29<00:00, 34.22it/s]


In [8]:
fid

[15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 15.1319188265338,
 11.302143232859633,
 11.302809950405535,
 11.303933828263895,
 11.305999848938457,
 11.307683157363678,
 11.310447651510113,
 11.313950178606836,
 11.318572871947673,
 11.324214153932587,
 11.331629828114535,
 11.302809950405535,
 11.305999848938457,
 11.310447651510113,
 11.318572871947673,
 11.331629828114535,
 11.35475002236505,
 11.40581665612791,
 11.630903824485848,
 10.876680464406702,
 11.172813527484443,
 11.303933926160603,
 11.310460632918703,
 11.324215347264953,
 11.354764816729533,
 11.465642899585593,
 10.87664636180802,
 11.211038945837117,
 11.272909884835538,
 11.32407247059518,
 11.411372986464174,
 11.305999848938457,
 11.318572871947673,
 11.35475002236505,
 11.630903824485848,
 11.172813527484443,
 11.272916355790482,
 11.345820078295631,
 11.576725740119805,
 10.8765490817543